In [1]:
import pandas as pd
import numpy as np
import datetime as DT
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score
import random

# Comenzamos con los datos de genero y edad

In [2]:
concatAge = pd.concat([pd.read_csv('data/datos_navent_fiuba/fiuba_2_postulantes_genero_y_edad.csv'),
                       pd.read_csv('data/entrega6/fiuba_2_postulantes_genero_y_edad.csv'),
                       pd.read_csv('data/Fiuba desde 15 Abril/fiuba_2_postulantes_genero_y_edad.csv')]
                    )
concatAge.head()

,idpostulante,fechanacimiento,sexo
0,NM5M,1970-12-03,FEM
1,5awk,1962-12-04,FEM
2,ZaO5,1978-08-10,FEM
3,NdJl,1969-05-09,MASC
4,eo2p,1981-02-16,MASC


In [3]:
concatAge.drop_duplicates('idpostulante',keep = 'last', inplace = True)
concatAge.reset_index(inplace = True)
concatAge.drop(columns = ['index'],inplace = True)
concatAge.rename(columns = {'fechanacimiento':'edad'}, inplace = True)


# hoy : 30/5/2018
hoy = pd.Timestamp(DT.datetime.now())
concatAge['edad'] = pd.to_datetime(concatAge['edad'],errors = 'coerce')
concatAge['edad'] = (hoy - concatAge['edad']).astype('<m8[Y]')
concatAge.head()

,idpostulante,edad,sexo
0,eo2p,37.0,MASC
1,1d2B,42.0,MASC
2,EBO0,44.0,FEM
3,a6MKW,43.0,MASC
4,6MWd4,43.0,MASC


In [4]:
def sexoABool(sexo):
    if (sexo == 'MASC'):
        return True
    if (sexo == 'FEM'):
        return False
    return (bool(random.getrandbits(1)))

In [5]:
concatAge['sexo'] = concatAge['sexo'].apply(sexoABool)
concatAge.rename(columns = {'sexo':'sexo_masculino'}, inplace = True)
concatAge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 504407 entries, 0 to 504406
Data columns (total 3 columns):
idpostulante      504407 non-null object
edad              478311 non-null float64
sexo_masculino    504407 non-null bool
dtypes: bool(1), float64(1), object(1)
memory usage: 8.2+ MB


In [6]:
concatAge['edad'].fillna(0,inplace = True)

# Pasamos a los datos sobre la educacion

In [7]:
concatEduc = pd.concat([pd.read_csv('data/datos_navent_fiuba/fiuba_1_postulantes_educacion.csv'),
                        pd.read_csv('data/entrega6/fiuba_1_postulantes_educacion.csv'),
                        pd.read_csv('data/Fiuba desde 15 Abril/fiuba_1_postulantes_educacion.csv')]
                      )
concatEduc.head()

,idpostulante,nombre,estado
0,NdJl,Posgrado,En Curso
1,8BkL,Universitario,En Curso
2,1d2B,Universitario,En Curso
3,NPBx,Universitario,En Curso
4,NPBx,Master,En Curso


In [8]:
concatEduc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1103193 entries, 0 to 397508
Data columns (total 3 columns):
idpostulante    1103193 non-null object
nombre          1103193 non-null object
estado          1103193 non-null object
dtypes: object(3)
memory usage: 33.7+ MB


In [9]:
concatEduc['nombre'].value_counts()

Universitario        404619
Secundario           382927
Terciario/Técnico    180091
Otro                  84674
Posgrado              33484
Master                16436
Doctorado               962
Name: nombre, dtype: int64

In [10]:
def estudioSuperior(nivel):
    if ((nivel=='Posgrado') or (nivel =='Doctorado') or (nivel=='Master')):
        return 1
    else:
        return 0

In [11]:
graduados = (concatEduc['estado'].astype(str)== 'Graduado').astype(int)

universitarios = (concatEduc['nombre'].astype(str) == 'Universitario').astype(int)

secundario = (concatEduc['nombre'].astype(str) == 'Secundario').astype(int)

mayor_a_universitario = concatEduc['nombre'].apply(estudioSuperior)

In [12]:
concatEduc['titulo_universitario'] = universitarios*graduados
concatEduc['titulo_secundario'] = secundario*graduados
concatEduc['titulo_superior'] = mayor_a_universitario * graduados

In [13]:
titulos = concatEduc.groupby('idpostulante').agg({'titulo_universitario':'max',
                                                   'titulo_secundario':'max',
                                                   'titulo_superior':'max'})
titulos.reset_index(inplace = True)

titulos.head()

,idpostulante,titulo_universitario,titulo_secundario,titulo_superior
0,0z5Dmrd,1,0,0
1,0z5JW1r,0,1,0
2,0z5VvGv,0,1,0
3,0zB01pE,0,0,0
4,0zB026d,0,1,0


In [14]:
usuarios = concatAge.merge(titulos, on = 'idpostulante', how = 'left')
usuarios.head()

,idpostulante,edad,sexo_masculino,titulo_universitario,titulo_secundario,titulo_superior
0,eo2p,37.0,True,1.0,1.0,1.0
1,1d2B,42.0,True,0.0,0.0,0.0
2,EBO0,44.0,False,1.0,0.0,1.0
3,a6MKW,43.0,True,1.0,0.0,1.0
4,6MWd4,43.0,True,1.0,0.0,0.0


In [15]:
usuarios['titulo_universitario'] = usuarios['titulo_universitario'].astype(bool)
usuarios['titulo_secundario'] = usuarios['titulo_secundario'].astype(bool)
usuarios['titulo_superior'] = usuarios['titulo_superior'].astype(bool)

In [16]:
usuarios.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 504407 entries, 0 to 504406
Data columns (total 6 columns):
idpostulante            504407 non-null object
edad                    504407 non-null float64
sexo_masculino          504407 non-null bool
titulo_universitario    504407 non-null bool
titulo_secundario       504407 non-null bool
titulo_superior         504407 non-null bool
dtypes: bool(4), float64(1), object(1)
memory usage: 13.5+ MB


# Pasamos a los datos sobre los avisos

In [17]:
avisos = pd.concat([pd.read_csv('data/datos_navent_fiuba/fiuba_6_avisos_detalle.csv'),
                    pd.read_csv('data/entrega6/fiuba_6_avisos_detalle.csv'),
                    pd.read_csv('data/Fiuba desde 15 Abril/fiuba_6_avisos_detalle.csv'),
                    pd.read_csv('data/fiuba_6_avisos_detalle_missing_nivel_laboral.csv')])


avisos.head()

,idaviso,idpais,titulo,descripcion,nombre_zona,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,8725750,1,VENDEDOR/A PROVINCIA DE SANTA FE,<p><strong><strong>Empresa:</strong></strong> ...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Comercial,VENTOR
1,17903700,1,Enfermeras,<p>Solicitamos para importante cadena de farma...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Salud,Farmacias Central Oeste
2,1000150677,1,Chofer de taxi,<p>TE GUSTA MANEJAR? QUERES GANAR PLATA HACIEN...,Capital Federal,NaN,Empedrado 2336,Full-time,Senior / Semi-Senior,Transporte,FAMITAX SRL
3,1000610287,1,CHOFER DE CAMIONETA BAHIA BLANCA - PUNTA ALTA,<p><strong>Somos una empresa multinacional que...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Transporte,Wurth Argentina S.A
4,1000872556,1,Operarios de Planta - Rubro Electrodomésticos,<p><strong>OPERARIOS DE PLANTA</strong></p><p>...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Producción,ELECTRO OUTLET SRL


In [18]:
avisos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45969 entries, 0 to 337
Data columns (total 11 columns):
idaviso                 45969 non-null int64
idpais                  45969 non-null int64
titulo                  45969 non-null object
descripcion             45969 non-null object
nombre_zona             45969 non-null object
ciudad                  225 non-null object
mapacalle               3444 non-null object
tipo_de_trabajo         45969 non-null object
nivel_laboral           45634 non-null object
nombre_area             45969 non-null object
denominacion_empresa    45955 non-null object
dtypes: int64(2), object(9)
memory usage: 4.2+ MB


In [19]:
avisos.drop_duplicates('idaviso', keep = 'last', inplace = True)
avisos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25288 entries, 4 to 337
Data columns (total 11 columns):
idaviso                 25288 non-null int64
idpais                  25288 non-null int64
titulo                  25288 non-null object
descripcion             25288 non-null object
nombre_zona             25288 non-null object
ciudad                  160 non-null object
mapacalle               2070 non-null object
tipo_de_trabajo         25288 non-null object
nivel_laboral           24953 non-null object
nombre_area             25288 non-null object
denominacion_empresa    25281 non-null object
dtypes: int64(2), object(9)
memory usage: 2.3+ MB


In [20]:
avisos.drop(columns = ['mapacalle','ciudad','idpais'], inplace = True)


In [21]:
avisos['tipo_de_trabajo'].value_counts()

Full-time          22831
Part-time           1746
Teletrabajo          248
Por Horas            125
Pasantia             119
Temporario            96
Por Contrato          88
Fines de Semana       28
Primer empleo          6
Voluntario             1
Name: tipo_de_trabajo, dtype: int64

In [22]:
avisos['tipo_de_trabajo'] = (avisos['tipo_de_trabajo'].astype(str) == 'Full-time').astype(bool)

In [23]:
avisos['nombre_zona'].value_counts()

Gran Buenos Aires              23017
Capital Federal                 2183
Buenos Aires (fuera de GBA)       57
GBA Oeste                         10
La Plata                           4
Mendoza                            3
Ciudad de Mendoza                  3
Cordoba                            2
Rosario                            2
Tucuman                            1
Corrientes                         1
Santa Cruz                         1
Santa Fe                           1
Neuquen                            1
San Juan                           1
Catamarca                          1
Name: nombre_zona, dtype: int64

In [24]:
avisos.rename(columns = {'nombre_zona':'GBA', 'tipo_de_trabajo':'Full-time'},inplace = True)

In [25]:
avisos['GBA'] = (avisos['GBA'].astype(str) == 'Gran Buenos Aires').astype(bool)
avisos.head(10)

,idaviso,titulo,descripcion,GBA,Full-time,nivel_laboral,nombre_area,denominacion_empresa
4,1000872556,Operarios de Planta - Rubro Electrodomésticos,<p><strong>OPERARIOS DE PLANTA</strong></p><p>...,True,True,Senior / Semi-Senior,Producción,ELECTRO OUTLET SRL
8,9240880,"Productores Asesores Independiente, para venta...",Agente\r\noficial Selecciona:</span></strong><...,False,True,Jefe / Supervisor / Responsable,Comercial,Agencia Oficial Alejandro Arizaga
19,1110185164,Administrativo de Recepción,<p>Centro Médico Accord se encuentra en la bús...,True,True,Junior,Administración,Unión Personal
21,1110513885,Distribuidor domiciliario con moto (SAN MIGUEL),<p>Importante correo privado ubicado en SAN MI...,True,True,Junior,Distribución,Asoko Tempo SA
32,1111034024,Vendedores para venta de medicina pre paga y o...,<p> </p><p><strong>Comercializadora incorpora ...,True,True,Senior / Semi-Senior,Comercial,JELS SRL
35,1111101289,PM. Zona Norte / Pacheco,<p>Acciona IT se encuentra en la búsqueda de P...,True,True,Senior / Semi-Senior,Liderazgo de Proyecto,ACCIONA IT
38,1111109704,"Supervisor de obra civil, eléctrico/ instrumen...",<p>Buscamos para importante empresa petrolera ...,True,True,Senior / Semi-Senior,Construcción,Hahn Solarz S.R.L.
45,1111172357,Empleado Administrativo para Tramites de Habil...,"<p style=""""><strong><em><span style="""">Brujula...",True,True,Senior / Semi-Senior,Administración,Brujula SA
46,1111174081,Gestor de Cobranzas - Telecobrador,<p><strong>En MAS ACTIVOS BPO te estamos esper...,False,False,Senior / Semi-Senior,Call Center,MAS ACTIVOS S.A.
47,1111235995,Analista de Recursos Humanos - Quilmes (Plazo...,<p>Cervecería y Maltería Quilmes es una de las...,True,True,Senior / Semi-Senior,Recursos Humanos,AB InBev - Cervecería y Maltería Quilmes


In [26]:
avisos['nivel_laboral'].value_counts()

Senior / Semi-Senior                    16975
Junior                                   4152
Otro                                     1977
Jefe / Supervisor / Responsable          1527
Gerencia / Alta Gerencia / Dirección      322
Name: nivel_laboral, dtype: int64

In [27]:
avisos.rename(columns = {'nivel_laboral':'nivel'}, inplace = True)


In [28]:
avisos['senior'] = (avisos['nivel'] == 'Senior/ Semi-Senior').astype(bool)
avisos['junior'] = (avisos['nivel'] == 'Junior').astype(bool)
avisos['otro'] = (avisos['nivel'] == 'Otro').astype(bool)
avisos['Jefe'] = (avisos['nivel'] == 'Jefe / Supervisor / Responsable').astype(bool)

In [29]:
def esChofer(titulo):
    if ('chofer' in titulo):
        return True
    if ('Chofer' in titulo):
        return True
    return False

In [30]:
avisos['chofer'] = avisos['titulo'].apply(esChofer)
avisos['chofer'].value_counts()

False    25113
True       175
Name: chofer, dtype: int64

In [31]:
avisos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25288 entries, 4 to 337
Data columns (total 13 columns):
idaviso                 25288 non-null int64
titulo                  25288 non-null object
descripcion             25288 non-null object
GBA                     25288 non-null bool
Full-time               25288 non-null bool
nivel                   24953 non-null object
nombre_area             25288 non-null object
denominacion_empresa    25281 non-null object
senior                  25288 non-null bool
junior                  25288 non-null bool
otro                    25288 non-null bool
Jefe                    25288 non-null bool
chofer                  25288 non-null bool
dtypes: bool(7), int64(1), object(5)
memory usage: 1.5+ MB


In [32]:
avisos['nombre_area'].value_counts().index

Index(['Ventas', 'Comercial', 'Administración', 'Producción', 'Programación',
       'Contabilidad', 'Tecnologia / Sistemas', 'Atención al Cliente',
       'Mantenimiento', 'Recursos Humanos',
       ...
       'Idiomas', 'Fonoaudiología', 'Farmacia hospitalaria',
       'Cuidados Intensivos Cardiologicos Adultos', 'Maternidad',
       'Esterilización', 'Diagnóstico por imágenes', 'Auditoría de Seguros',
       'Prácticas cardiológicas', 'Instrumentación Minera'],
      dtype='object', length=188)

In [33]:
top_areas = avisos['nombre_area'].value_counts().nlargest(10).index
for area in top_areas:
    avisos[area] = (avisos['nombre_area'] == area).astype(bool)
    
avisos.head()

,idaviso,titulo,descripcion,GBA,Full-time,nivel,nombre_area,denominacion_empresa,senior,junior,...,Ventas,Comercial,Administración,Producción,Programación,Contabilidad,Tecnologia / Sistemas,Atención al Cliente,Mantenimiento,Recursos Humanos
4,1000872556,Operarios de Planta - Rubro Electrodomésticos,<p><strong>OPERARIOS DE PLANTA</strong></p><p>...,True,True,Senior / Semi-Senior,Producción,ELECTRO OUTLET SRL,False,False,...,False,False,False,True,False,False,False,False,False,False
8,9240880,"Productores Asesores Independiente, para venta...",Agente\r\noficial Selecciona:</span></strong><...,False,True,Jefe / Supervisor / Responsable,Comercial,Agencia Oficial Alejandro Arizaga,False,False,...,False,True,False,False,False,False,False,False,False,False
19,1110185164,Administrativo de Recepción,<p>Centro Médico Accord se encuentra en la bús...,True,True,Junior,Administración,Unión Personal,False,True,...,False,False,True,False,False,False,False,False,False,False
21,1110513885,Distribuidor domiciliario con moto (SAN MIGUEL),<p>Importante correo privado ubicado en SAN MI...,True,True,Junior,Distribución,Asoko Tempo SA,False,True,...,False,False,False,False,False,False,False,False,False,False
32,1111034024,Vendedores para venta de medicina pre paga y o...,<p> </p><p><strong>Comercializadora incorpora ...,True,True,Senior / Semi-Senior,Comercial,JELS SRL,False,False,...,False,True,False,False,False,False,False,False,False,False


In [34]:
avisos.drop(columns = ['titulo','descripcion','denominacion_empresa','nivel'],
            inplace = True)

In [48]:
avisos.head()

,idaviso,GBA,Full-time,nombre_area,senior,junior,otro,Jefe,chofer,Ventas,Comercial,Administración,Producción,Programación,Contabilidad,Tecnologia / Sistemas,Atención al Cliente,Mantenimiento,Recursos Humanos
4,1000872556,True,True,Producción,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False
8,9240880,False,True,Comercial,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False
19,1110185164,True,True,Administración,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False
21,1110513885,True,True,Distribución,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False
32,1111034024,True,True,Comercial,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False


# Vistas

In [35]:
concatVistas = pd.concat([pd.read_csv('data/datos_navent_fiuba/fiuba_3_vistas.csv'),
                          pd.read_csv('data/entrega6/fiuba_3_vistas.csv'),
                          pd.read_csv('data/Fiuba desde 15 Abril/fiuba_3_vistas.csv')])

concatVistas.head()

,idAviso,timestamp,idpostulante
0,1111780242,2018-02-23T13:38:13.187-0500,YjVJQ6Z
1,1112263876,2018-02-23T13:38:14.296-0500,BmVpYoR
2,1112327963,2018-02-23T13:38:14.329-0500,wVkBzZd
3,1112318643,2018-02-23T13:38:17.921-0500,OqmP9pv
4,1111903673,2018-02-23T13:38:18.973-0500,DrpbXDP


In [36]:
concatVistas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18368813 entries, 0 to 11648229
Data columns (total 3 columns):
idAviso         int64
timestamp       object
idpostulante    object
dtypes: int64(1), object(2)
memory usage: 560.6+ MB


In [37]:
concatVistas['visita'] = True
concatVistas.rename(columns = {'idAviso':'idaviso'}, inplace = True)
concatVistas.drop(columns = 'timestamp',inplace = True)
concatVistas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18368813 entries, 0 to 11648229
Data columns (total 3 columns):
idaviso         int64
idpostulante    object
visita          bool
dtypes: bool(1), int64(1), object(1)
memory usage: 437.9+ MB


In [38]:
concatVistas.drop_duplicates(subset = ['idaviso','idpostulante'],
                             inplace = True, keep = 'first')

In [39]:
concatVistas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8597915 entries, 0 to 11648221
Data columns (total 3 columns):
idaviso         int64
idpostulante    object
visita          bool
dtypes: bool(1), int64(1), object(1)
memory usage: 205.0+ MB


# Pasamos a las postulaciones

In [40]:
postulaciones = pd.concat([pd.read_csv('data/datos_navent_fiuba/fiuba_4_postulaciones.csv'),
                           pd.read_csv('data/entrega6/fiuba_4_postulaciones.csv')])
postulaciones.head()

,idaviso,idpostulante,fechapostulacion
0,1112257047,NM5M,2018-01-15 16:22:34
1,1111920714,NM5M,2018-02-06 09:04:50
2,1112346945,NM5M,2018-02-22 09:04:47
3,1112345547,NM5M,2018-02-22 09:04:59
4,1112237522,5awk,2018-01-25 18:55:03


In [41]:
postulaciones.drop(columns = 'fechapostulacion', inplace = True)

In [42]:
postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8311264 entries, 0 to 4909640
Data columns (total 2 columns):
idaviso         int64
idpostulante    object
dtypes: int64(1), object(1)
memory usage: 190.2+ MB


In [43]:
postulaciones.drop_duplicates(subset = ['idaviso','idpostulante'],
                              keep = 'first',
                              inplace = True)
postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6603752 entries, 0 to 4909640
Data columns (total 2 columns):
idaviso         int64
idpostulante    object
dtypes: int64(1), object(1)
memory usage: 151.1+ MB


In [44]:
postulaciones = postulaciones.merge(usuarios,on='idpostulante',how='left')
postulaciones.head()

,idaviso,idpostulante,edad,sexo_masculino,titulo_universitario,titulo_secundario,titulo_superior
0,1112257047,NM5M,47.0,False,False,True,False
1,1111920714,NM5M,47.0,False,False,True,False
2,1112346945,NM5M,47.0,False,False,True,False
3,1112345547,NM5M,47.0,False,False,True,False
4,1112237522,5awk,55.0,False,True,False,False


In [45]:
postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6603752 entries, 0 to 6603751
Data columns (total 7 columns):
idaviso                 int64
idpostulante            object
edad                    float64
sexo_masculino          bool
titulo_universitario    bool
titulo_secundario       bool
titulo_superior         bool
dtypes: bool(4), float64(1), int64(1), object(1)
memory usage: 226.7+ MB


In [46]:
postulaciones = postulaciones.merge(avisos[['idaviso','nombre_area']],
                                    on = 'idaviso',
                                    how = 'left')

In [47]:
postulaciones.head()

,idaviso,idpostulante,edad,sexo_masculino,titulo_universitario,titulo_secundario,titulo_superior,nombre_area
0,1112257047,NM5M,47.0,False,False,True,False,Atención al Cliente
1,1111920714,NM5M,47.0,False,False,True,False,Telemarketing
2,1112346945,NM5M,47.0,False,False,True,False,Telemarketing
3,1112345547,NM5M,47.0,False,False,True,False,Telemarketing
4,1112237522,5awk,55.0,False,True,False,False,Contabilidad


### Aca agrego features a los usuarios de acuerdo a las areas a las que postularon

In [49]:
areas_por_postulante = postulaciones.groupby('idpostulante')\
                .agg({'nombre_area': lambda x: list(x),
                      'idaviso':'count'})

areas_por_postulante.head()

,nombre_area,idaviso
idpostulante,,
0z5Dmrd,"[Recursos Humanos, Recursos Humanos]",2
0z5JW1r,"[Almacén / Depósito / Expedición, Comercial, D...",7
0z5VvGv,"[Ventas, Ventas, Atención al Cliente, Atención...",50
0zB01pE,"[Administracion de Seguros, Administracion de ...",3
0zB026d,"[Producción, Producción, Producción, Producció...",10


In [50]:
areas_por_postulante.reset_index(inplace = True)
areas_por_postulante.rename(columns = {'nombre_area':'areas', 'idaviso':'cantidad_postulaciones'},
                            inplace = True)

areas_por_postulante.head()

,idpostulante,areas,cantidad_postulaciones
0,0z5Dmrd,"[Recursos Humanos, Recursos Humanos]",2
1,0z5JW1r,"[Almacén / Depósito / Expedición, Comercial, D...",7
2,0z5VvGv,"[Ventas, Ventas, Atención al Cliente, Atención...",50
3,0zB01pE,"[Administracion de Seguros, Administracion de ...",3
4,0zB026d,"[Producción, Producción, Producción, Producció...",10


In [51]:
areas_por_postulante.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 348883 entries, 0 to 348882
Data columns (total 3 columns):
idpostulante              348883 non-null object
areas                     348883 non-null object
cantidad_postulaciones    348883 non-null int64
dtypes: int64(1), object(2)
memory usage: 8.0+ MB


In [52]:
for area in top_areas:
    print(area)
    lista_area = []
    for i in range(0,len(areas_por_postulante)):
        lista_areas_usuario = areas_por_postulante.loc[i,'areas']
        lista_area.append(lista_areas_usuario.count(area))
    
    areas_por_postulante['postulaciones_'+ area] = pd.Series(lista_area)

Ventas
Comercial
Administración
Producción
Programación
Contabilidad
Tecnologia / Sistemas
Atención al Cliente
Mantenimiento
Recursos Humanos


In [53]:
areas_por_postulante.head()

,idpostulante,areas,cantidad_postulaciones,postulaciones_Ventas,postulaciones_Comercial,postulaciones_Administración,postulaciones_Producción,postulaciones_Programación,postulaciones_Contabilidad,postulaciones_Tecnologia / Sistemas,postulaciones_Atención al Cliente,postulaciones_Mantenimiento,postulaciones_Recursos Humanos
0,0z5Dmrd,"[Recursos Humanos, Recursos Humanos]",2,0,0,0,0,0,0,0,0,0,2
1,0z5JW1r,"[Almacén / Depósito / Expedición, Comercial, D...",7,1,1,0,0,0,0,0,0,0,0
2,0z5VvGv,"[Ventas, Ventas, Atención al Cliente, Atención...",50,13,7,1,0,0,0,1,11,0,0
3,0zB01pE,"[Administracion de Seguros, Administracion de ...",3,0,0,0,0,0,0,0,0,0,0
4,0zB026d,"[Producción, Producción, Producción, Producció...",10,1,0,0,8,0,0,0,0,0,0


In [54]:
usuarios = usuarios.merge(areas_por_postulante.drop(columns = 'areas'),
                          on = 'idpostulante',
                          how = 'left')
usuarios.head()

,idpostulante,edad,sexo_masculino,titulo_universitario,titulo_secundario,titulo_superior,cantidad_postulaciones,postulaciones_Ventas,postulaciones_Comercial,postulaciones_Administración,postulaciones_Producción,postulaciones_Programación,postulaciones_Contabilidad,postulaciones_Tecnologia / Sistemas,postulaciones_Atención al Cliente,postulaciones_Mantenimiento,postulaciones_Recursos Humanos
0,eo2p,37.0,True,True,True,True,6.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1d2B,42.0,True,False,False,False,4.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,EBO0,44.0,False,True,False,True,8.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,a6MKW,43.0,True,True,False,True,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,6MWd4,43.0,True,True,False,False,3.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [55]:
usuarios.isnull().sum()

idpostulante                                0
edad                                        0
sexo_masculino                              0
titulo_universitario                        0
titulo_secundario                           0
titulo_superior                             0
cantidad_postulaciones                 155524
postulaciones_Ventas                   155524
postulaciones_Comercial                155524
postulaciones_Administración           155524
postulaciones_Producción               155524
postulaciones_Programación             155524
postulaciones_Contabilidad             155524
postulaciones_Tecnologia / Sistemas    155524
postulaciones_Atención al Cliente      155524
postulaciones_Mantenimiento            155524
postulaciones_Recursos Humanos         155524
dtype: int64

In [56]:
usuarios.fillna(0, inplace = True)
usuarios.isnull().sum()

idpostulante                           0
edad                                   0
sexo_masculino                         0
titulo_universitario                   0
titulo_secundario                      0
titulo_superior                        0
cantidad_postulaciones                 0
postulaciones_Ventas                   0
postulaciones_Comercial                0
postulaciones_Administración           0
postulaciones_Producción               0
postulaciones_Programación             0
postulaciones_Contabilidad             0
postulaciones_Tecnologia / Sistemas    0
postulaciones_Atención al Cliente      0
postulaciones_Mantenimiento            0
postulaciones_Recursos Humanos         0
dtype: int64

In [57]:
postulaciones.drop(columns = 'nombre_area', inplace = True)
postulaciones = postulaciones.merge(avisos, 
                                    on = 'idaviso',
                                    how = 'left')

postulaciones.head()

,idaviso,idpostulante,edad,sexo_masculino,titulo_universitario,titulo_secundario,titulo_superior,GBA,Full-time,nombre_area,...,Ventas,Comercial,Administración,Producción,Programación,Contabilidad,Tecnologia / Sistemas,Atención al Cliente,Mantenimiento,Recursos Humanos
0,1112257047,NM5M,47.0,False,False,True,False,True,False,Atención al Cliente,...,False,False,False,False,False,False,False,True,False,False
1,1111920714,NM5M,47.0,False,False,True,False,True,False,Telemarketing,...,False,False,False,False,False,False,False,False,False,False
2,1112346945,NM5M,47.0,False,False,True,False,True,True,Telemarketing,...,False,False,False,False,False,False,False,False,False,False
3,1112345547,NM5M,47.0,False,False,True,False,True,True,Telemarketing,...,False,False,False,False,False,False,False,False,False,False
4,1112237522,5awk,55.0,False,True,False,False,True,True,Contabilidad,...,False,False,False,False,False,True,False,False,False,False


In [58]:
postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6603752 entries, 0 to 6603751
Data columns (total 25 columns):
idaviso                  int64
idpostulante             object
edad                     float64
sexo_masculino           bool
titulo_universitario     bool
titulo_secundario        bool
titulo_superior          bool
GBA                      object
Full-time                object
nombre_area              object
senior                   object
junior                   object
otro                     object
Jefe                     object
chofer                   object
Ventas                   object
Comercial                object
Administración           object
Producción               object
Programación             object
Contabilidad             object
Tecnologia / Sistemas    object
Atención al Cliente      object
Mantenimiento            object
Recursos Humanos         object
dtypes: bool(4), float64(1), int64(1), object(19)
memory usage: 1.1+ GB


In [59]:
postulaciones['postulacion'] = True
postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6603752 entries, 0 to 6603751
Data columns (total 26 columns):
idaviso                  int64
idpostulante             object
edad                     float64
sexo_masculino           bool
titulo_universitario     bool
titulo_secundario        bool
titulo_superior          bool
GBA                      object
Full-time                object
nombre_area              object
senior                   object
junior                   object
otro                     object
Jefe                     object
chofer                   object
Ventas                   object
Comercial                object
Administración           object
Producción               object
Programación             object
Contabilidad             object
Tecnologia / Sistemas    object
Atención al Cliente      object
Mantenimiento            object
Recursos Humanos         object
postulacion              bool
dtypes: bool(5), float64(1), int64(1), object(19)
memory usage: 1

In [62]:
columnas_booleanas = postulaciones.columns[3:26]
for columna in columnas_booleanas:
    postulaciones[columna] = postulaciones[columna].astype(bool)
    
postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6603752 entries, 0 to 6603751
Data columns (total 26 columns):
idaviso                  int64
idpostulante             object
edad                     float64
sexo_masculino           bool
titulo_universitario     bool
titulo_secundario        bool
titulo_superior          bool
GBA                      bool
Full-time                bool
nombre_area              bool
senior                   bool
junior                   bool
otro                     bool
Jefe                     bool
chofer                   bool
Ventas                   bool
Comercial                bool
Administración           bool
Producción               bool
Programación             bool
Contabilidad             bool
Tecnologia / Sistemas    bool
Atención al Cliente      bool
Mantenimiento            bool
Recursos Humanos         bool
postulacion              bool
dtypes: bool(23), float64(1), int64(1), object(1)
memory usage: 346.4+ MB


In [63]:
postulaciones = postulaciones.merge(concatVistas,
                                    on = ['idaviso','idpostulante'],
                                    how = 'left')
postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6603752 entries, 0 to 6603751
Data columns (total 27 columns):
idaviso                  int64
idpostulante             object
edad                     float64
sexo_masculino           bool
titulo_universitario     bool
titulo_secundario        bool
titulo_superior          bool
GBA                      bool
Full-time                bool
nombre_area              bool
senior                   bool
junior                   bool
otro                     bool
Jefe                     bool
chofer                   bool
Ventas                   bool
Comercial                bool
Administración           bool
Producción               bool
Programación             bool
Contabilidad             bool
Tecnologia / Sistemas    bool
Atención al Cliente      bool
Mantenimiento            bool
Recursos Humanos         bool
postulacion              bool
visita                   object
dtypes: bool(23), float64(1), int64(1), object(2)
memory usage: 396.8

In [64]:
postulaciones['visita'].fillna(False,inplace = True)
postulaciones.head()

,idaviso,idpostulante,edad,sexo_masculino,titulo_universitario,titulo_secundario,titulo_superior,GBA,Full-time,nombre_area,...,Administración,Producción,Programación,Contabilidad,Tecnologia / Sistemas,Atención al Cliente,Mantenimiento,Recursos Humanos,postulacion,visita
0,1112257047,NM5M,47.0,False,False,True,False,True,False,True,...,False,False,False,False,False,True,False,False,True,False
1,1111920714,NM5M,47.0,False,False,True,False,True,False,True,...,False,False,False,False,False,False,False,False,True,False
2,1112346945,NM5M,47.0,False,False,True,False,True,True,True,...,False,False,False,False,False,False,False,False,True,False
3,1112345547,NM5M,47.0,False,False,True,False,True,True,True,...,False,False,False,False,False,False,False,False,True,False
4,1112237522,5awk,55.0,False,True,False,False,True,True,True,...,False,False,False,True,False,False,False,False,True,False


In [65]:
postulaciones['visita'] = postulaciones['visita'].astype(bool)
postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6603752 entries, 0 to 6603751
Data columns (total 27 columns):
idaviso                  int64
idpostulante             object
edad                     float64
sexo_masculino           bool
titulo_universitario     bool
titulo_secundario        bool
titulo_superior          bool
GBA                      bool
Full-time                bool
nombre_area              bool
senior                   bool
junior                   bool
otro                     bool
Jefe                     bool
chofer                   bool
Ventas                   bool
Comercial                bool
Administración           bool
Producción               bool
Programación             bool
Contabilidad             bool
Tecnologia / Sistemas    bool
Atención al Cliente      bool
Mantenimiento            bool
Recursos Humanos         bool
postulacion              bool
visita                   bool
dtypes: bool(24), float64(1), int64(1), object(1)
memory usage: 352.7+ 

In [66]:
postulaciones.isnull().sum()

idaviso                  0
idpostulante             0
edad                     0
sexo_masculino           0
titulo_universitario     0
titulo_secundario        0
titulo_superior          0
GBA                      0
Full-time                0
nombre_area              0
senior                   0
junior                   0
otro                     0
Jefe                     0
chofer                   0
Ventas                   0
Comercial                0
Administración           0
Producción               0
Programación             0
Contabilidad             0
Tecnologia / Sistemas    0
Atención al Cliente      0
Mantenimiento            0
Recursos Humanos         0
postulacion              0
visita                   0
dtype: int64

In [70]:
postulaciones = postulaciones.merge(usuarios, on = 'idpostulante', how = 'left')
postulaciones.head()

,idaviso,idpostulante,edad_x,sexo_masculino_x,titulo_universitario_x,titulo_secundario_x,titulo_superior_x,GBA,Full-time,nombre_area,...,postulaciones_Ventas,postulaciones_Comercial,postulaciones_Administración,postulaciones_Producción,postulaciones_Programación,postulaciones_Contabilidad,postulaciones_Tecnologia / Sistemas,postulaciones_Atención al Cliente,postulaciones_Mantenimiento,postulaciones_Recursos Humanos
0,1112257047,NM5M,47.0,False,False,True,False,True,False,True,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,1111920714,NM5M,47.0,False,False,True,False,True,False,True,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1112346945,NM5M,47.0,False,False,True,False,True,True,True,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,1112345547,NM5M,47.0,False,False,True,False,True,True,True,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,1112237522,5awk,55.0,False,True,False,False,True,True,True,...,0.0,0.0,4.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0


In [72]:
postulaciones.drop(columns = postulaciones.columns[2:6], inplace = True)

In [73]:
postulaciones.rename(columns = {'edad_y':'edad','sexo_masculino_y':'sexo_masculino',
                                'titulo_universitario_y':'titulo_universitario',
                                'titulo_secundario_y':'titulo_secundario',
                                'titulo_superior_y':'titulo_superior'}, inplace = True)
postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6603752 entries, 0 to 6603751
Data columns (total 39 columns):
idaviso                                int64
idpostulante                           object
titulo_superior_x                      bool
GBA                                    bool
Full-time                              bool
nombre_area                            bool
senior                                 bool
junior                                 bool
otro                                   bool
Jefe                                   bool
chofer                                 bool
Ventas                                 bool
Comercial                              bool
Administración                         bool
Producción                             bool
Programación                           bool
Contabilidad                           bool
Tecnologia / Sistemas                  bool
Atención al Cliente                    bool
Mantenimiento                          bool
Recu

In [75]:
postulaciones.drop(columns = 'titulo_superior_x', inplace = True)
postulaciones.columns

Index(['idaviso', 'idpostulante', 'GBA', 'Full-time', 'nombre_area', 'senior',
       'junior', 'otro', 'Jefe', 'chofer', 'Ventas', 'Comercial',
       'Administración', 'Producción', 'Programación', 'Contabilidad',
       'Tecnologia / Sistemas', 'Atención al Cliente', 'Mantenimiento',
       'Recursos Humanos', 'postulacion', 'visita', 'edad', 'sexo_masculino',
       'titulo_universitario', 'titulo_secundario', 'titulo_superior',
       'cantidad_postulaciones', 'postulaciones_Ventas',
       'postulaciones_Comercial', 'postulaciones_Administración',
       'postulaciones_Producción', 'postulaciones_Programación',
       'postulaciones_Contabilidad', 'postulaciones_Tecnologia / Sistemas',
       'postulaciones_Atención al Cliente', 'postulaciones_Mantenimiento',
       'postulaciones_Recursos Humanos'],
      dtype='object')

In [86]:
postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6603752 entries, 0 to 6603751
Data columns (total 38 columns):
idaviso                                int64
idpostulante                           object
GBA                                    bool
Full-time                              bool
nombre_area                            bool
senior                                 bool
junior                                 bool
otro                                   bool
Jefe                                   bool
chofer                                 bool
Ventas                                 bool
Comercial                              bool
Administración                         bool
Producción                             bool
Programación                           bool
Contabilidad                           bool
Tecnologia / Sistemas                  bool
Atención al Cliente                    bool
Mantenimiento                          bool
Recursos Humanos                       bool
post

In [87]:
postulaciones.drop(columns = 'nombre_area', inplace = True)

In [88]:
postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6603752 entries, 0 to 6603751
Data columns (total 37 columns):
idaviso                                int64
idpostulante                           object
GBA                                    bool
Full-time                              bool
senior                                 bool
junior                                 bool
otro                                   bool
Jefe                                   bool
chofer                                 bool
Ventas                                 bool
Comercial                              bool
Administración                         bool
Producción                             bool
Programación                           bool
Contabilidad                           bool
Tecnologia / Sistemas                  bool
Atención al Cliente                    bool
Mantenimiento                          bool
Recursos Humanos                       bool
postulacion                            bool
visi

In [67]:
no_postulaciones = usuarios.sample(10000)
for i in range(1,200):
    no_postulaciones = pd.concat([no_postulaciones, usuarios.sample(20000)])

In [68]:
avisos_no_postulaciones = avisos.sample(10000)
for i in range(1,200):
    avisos_no_postulaciones = pd.concat([avisos_no_postulaciones,avisos.sample(20000)])

In [69]:
no_postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3990000 entries, 476178 to 418342
Data columns (total 17 columns):
idpostulante                           object
edad                                   float64
sexo_masculino                         bool
titulo_universitario                   bool
titulo_secundario                      bool
titulo_superior                        bool
cantidad_postulaciones                 float64
postulaciones_Ventas                   float64
postulaciones_Comercial                float64
postulaciones_Administración           float64
postulaciones_Producción               float64
postulaciones_Programación             float64
postulaciones_Contabilidad             float64
postulaciones_Tecnologia / Sistemas    float64
postulaciones_Atención al Cliente      float64
postulaciones_Mantenimiento            float64
postulaciones_Recursos Humanos         float64
dtypes: bool(4), float64(12), object(1)
memory usage: 441.4+ MB


In [76]:
no_postulaciones.head()

,idpostulante,edad,sexo_masculino,titulo_universitario,titulo_secundario,titulo_superior,cantidad_postulaciones,postulaciones_Ventas,postulaciones_Comercial,postulaciones_Administración,postulaciones_Producción,postulaciones_Programación,postulaciones_Contabilidad,postulaciones_Tecnologia / Sistemas,postulaciones_Atención al Cliente,postulaciones_Mantenimiento,postulaciones_Recursos Humanos
476178,ak5ld6E,36.0,False,False,False,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50474,5mqORRZ,20.0,False,False,True,False,5.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
454921,2zP9OAY,24.0,False,True,True,True,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
402898,ak5Mwm9,33.0,True,False,True,False,4.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
346291,A3vVNPG,27.0,True,False,False,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [77]:
no_postulaciones.reset_index(inplace = True)
no_postulaciones.drop(columns = 'index', inplace = True)
no_postulaciones

,idpostulante,edad,sexo_masculino,titulo_universitario,titulo_secundario,titulo_superior,cantidad_postulaciones,postulaciones_Ventas,postulaciones_Comercial,postulaciones_Administración,postulaciones_Producción,postulaciones_Programación,postulaciones_Contabilidad,postulaciones_Tecnologia / Sistemas,postulaciones_Atención al Cliente,postulaciones_Mantenimiento,postulaciones_Recursos Humanos
0,ak5ld6E,36.0,False,False,False,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5mqORRZ,20.0,False,False,True,False,5.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2zP9OAY,24.0,False,True,True,True,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ak5Mwm9,33.0,True,False,True,False,4.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
4,A3vVNPG,27.0,True,False,False,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,ZD2bL1V,37.0,False,True,True,True,3.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,ow2bB9e,21.0,False,True,False,False,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,VNxol2j,19.0,True,False,True,False,91.0,29.0,6.0,7.0,2.0,0.0,1.0,0.0,5.0,1.0,0.0
8,W9APDxN,22.0,True,False,True,False,3.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,zvaekVk,0.0,True,False,False,False,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [78]:
avisos_no_postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3990000 entries, 4155 to 7963
Data columns (total 19 columns):
idaviso                  int64
GBA                      bool
Full-time                bool
nombre_area              object
senior                   bool
junior                   bool
otro                     bool
Jefe                     bool
chofer                   bool
Ventas                   bool
Comercial                bool
Administración           bool
Producción               bool
Programación             bool
Contabilidad             bool
Tecnologia / Sistemas    bool
Atención al Cliente      bool
Mantenimiento            bool
Recursos Humanos         bool
dtypes: bool(17), int64(1), object(1)
memory usage: 156.0+ MB


In [79]:
avisos_no_postulaciones.head()

,idaviso,GBA,Full-time,nombre_area,senior,junior,otro,Jefe,chofer,Ventas,Comercial,Administración,Producción,Programación,Contabilidad,Tecnologia / Sistemas,Atención al Cliente,Mantenimiento,Recursos Humanos
4155,1112291730,True,True,Mantenimiento y Limpieza,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
9498,1112492601,True,True,Ingeniería Electromecánica,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4219,1112226443,True,True,Desarrollo de Negocios,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
8283,1112241656,True,True,Cadetería,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2572,1112220304,True,True,Medicina,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False


In [80]:
avisos_no_postulaciones.reset_index(inplace = True)

avisos_no_postulaciones.head()

,index,idaviso,GBA,Full-time,nombre_area,senior,junior,otro,Jefe,chofer,Ventas,Comercial,Administración,Producción,Programación,Contabilidad,Tecnologia / Sistemas,Atención al Cliente,Mantenimiento,Recursos Humanos
0,4155,1112291730,True,True,Mantenimiento y Limpieza,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,9498,1112492601,True,True,Ingeniería Electromecánica,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,4219,1112226443,True,True,Desarrollo de Negocios,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,8283,1112241656,True,True,Cadetería,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,2572,1112220304,True,True,Medicina,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False


In [82]:
avisos_no_postulaciones.drop(columns = 'index', inplace = True)

In [83]:
avisos_no_postulaciones.head()

,idaviso,GBA,Full-time,senior,junior,otro,Jefe,chofer,Ventas,Comercial,Administración,Producción,Programación,Contabilidad,Tecnologia / Sistemas,Atención al Cliente,Mantenimiento,Recursos Humanos
0,1112291730,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,1112492601,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,1112226443,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,1112241656,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,1112220304,True,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False


In [84]:
avisos_no_postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3990000 entries, 0 to 3989999
Data columns (total 18 columns):
idaviso                  int64
GBA                      bool
Full-time                bool
senior                   bool
junior                   bool
otro                     bool
Jefe                     bool
chofer                   bool
Ventas                   bool
Comercial                bool
Administración           bool
Producción               bool
Programación             bool
Contabilidad             bool
Tecnologia / Sistemas    bool
Atención al Cliente      bool
Mantenimiento            bool
Recursos Humanos         bool
dtypes: bool(17), int64(1)
memory usage: 95.1 MB


In [85]:
no_postulaciones = no_postulaciones.join(avisos_no_postulaciones)
no_postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3990000 entries, 0 to 3989999
Data columns (total 35 columns):
idpostulante                           object
edad                                   float64
sexo_masculino                         bool
titulo_universitario                   bool
titulo_secundario                      bool
titulo_superior                        bool
cantidad_postulaciones                 float64
postulaciones_Ventas                   float64
postulaciones_Comercial                float64
postulaciones_Administración           float64
postulaciones_Producción               float64
postulaciones_Programación             float64
postulaciones_Contabilidad             float64
postulaciones_Tecnologia / Sistemas    float64
postulaciones_Atención al Cliente      float64
postulaciones_Mantenimiento            float64
postulaciones_Recursos Humanos         float64
idaviso                                int64
GBA                                    bool
Full-time   

In [89]:
no_postulaciones['postulacion'] = False
no_postulaciones.head()

,idpostulante,edad,sexo_masculino,titulo_universitario,titulo_secundario,titulo_superior,cantidad_postulaciones,postulaciones_Ventas,postulaciones_Comercial,postulaciones_Administración,...,Comercial,Administración,Producción,Programación,Contabilidad,Tecnologia / Sistemas,Atención al Cliente,Mantenimiento,Recursos Humanos,postulacion
0,ak5ld6E,36.0,False,False,False,False,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
1,5mqORRZ,20.0,False,False,True,False,5.0,2.0,2.0,0.0,...,False,False,False,False,False,False,False,False,False,False
2,2zP9OAY,24.0,False,True,True,True,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
3,ak5Mwm9,33.0,True,False,True,False,4.0,0.0,2.0,0.0,...,False,False,False,False,False,False,False,False,False,False
4,A3vVNPG,27.0,True,False,False,False,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False


In [90]:
no_postulaciones = no_postulaciones.merge(concatVistas,
                                          on = ['idaviso','idpostulante'],
                                          how = 'left')
no_postulaciones.head()

,idpostulante,edad,sexo_masculino,titulo_universitario,titulo_secundario,titulo_superior,cantidad_postulaciones,postulaciones_Ventas,postulaciones_Comercial,postulaciones_Administración,...,Administración,Producción,Programación,Contabilidad,Tecnologia / Sistemas,Atención al Cliente,Mantenimiento,Recursos Humanos,postulacion,visita
0,ak5ld6E,36.0,False,False,False,False,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,NaN
1,5mqORRZ,20.0,False,False,True,False,5.0,2.0,2.0,0.0,...,False,False,False,False,False,False,False,False,False,NaN
2,2zP9OAY,24.0,False,True,True,True,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,NaN
3,ak5Mwm9,33.0,True,False,True,False,4.0,0.0,2.0,0.0,...,False,False,False,False,False,False,False,False,False,NaN
4,A3vVNPG,27.0,True,False,False,False,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,NaN


In [91]:
no_postulaciones['visita'].fillna(False, inplace = True)
no_postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3990000 entries, 0 to 3989999
Data columns (total 37 columns):
idpostulante                           object
edad                                   float64
sexo_masculino                         bool
titulo_universitario                   bool
titulo_secundario                      bool
titulo_superior                        bool
cantidad_postulaciones                 float64
postulaciones_Ventas                   float64
postulaciones_Comercial                float64
postulaciones_Administración           float64
postulaciones_Producción               float64
postulaciones_Programación             float64
postulaciones_Contabilidad             float64
postulaciones_Tecnologia / Sistemas    float64
postulaciones_Atención al Cliente      float64
postulaciones_Mantenimiento            float64
postulaciones_Recursos Humanos         float64
idaviso                                int64
GBA                                    bool
Full-time   

In [92]:
no_postulaciones = no_postulaciones[postulaciones.columns]
no_postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3990000 entries, 0 to 3989999
Data columns (total 37 columns):
idaviso                                int64
idpostulante                           object
GBA                                    bool
Full-time                              bool
senior                                 bool
junior                                 bool
otro                                   bool
Jefe                                   bool
chofer                                 bool
Ventas                                 bool
Comercial                              bool
Administración                         bool
Producción                             bool
Programación                           bool
Contabilidad                           bool
Tecnologia / Sistemas                  bool
Atención al Cliente                    bool
Mantenimiento                          bool
Recursos Humanos                       bool
postulacion                            bool
visi

In [93]:
postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6603752 entries, 0 to 6603751
Data columns (total 37 columns):
idaviso                                int64
idpostulante                           object
GBA                                    bool
Full-time                              bool
senior                                 bool
junior                                 bool
otro                                   bool
Jefe                                   bool
chofer                                 bool
Ventas                                 bool
Comercial                              bool
Administración                         bool
Producción                             bool
Programación                           bool
Contabilidad                           bool
Tecnologia / Sistemas                  bool
Atención al Cliente                    bool
Mantenimiento                          bool
Recursos Humanos                       bool
postulacion                            bool
visi

In [95]:
postulaciones.isnull().sum()

idaviso                                0
idpostulante                           0
GBA                                    0
Full-time                              0
senior                                 0
junior                                 0
otro                                   0
Jefe                                   0
chofer                                 0
Ventas                                 0
Comercial                              0
Administración                         0
Producción                             0
Programación                           0
Contabilidad                           0
Tecnologia / Sistemas                  0
Atención al Cliente                    0
Mantenimiento                          0
Recursos Humanos                       0
postulacion                            0
visita                                 0
edad                                   0
sexo_masculino                         0
titulo_universitario                   0
titulo_secundari

In [96]:
postulaciones = pd.concat([postulaciones,no_postulaciones])

In [97]:
postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10593752 entries, 0 to 3989999
Data columns (total 37 columns):
idaviso                                int64
idpostulante                           object
GBA                                    bool
Full-time                              bool
senior                                 bool
junior                                 bool
otro                                   bool
Jefe                                   bool
chofer                                 bool
Ventas                                 bool
Comercial                              bool
Administración                         bool
Producción                             bool
Programación                           bool
Contabilidad                           bool
Tecnologia / Sistemas                  bool
Atención al Cliente                    bool
Mantenimiento                          bool
Recursos Humanos                       bool
postulacion                            bool
vis

In [98]:
postulaciones.drop_duplicates(subset = ['idaviso','idpostulante'], keep = 'first',inplace = True)
postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10591186 entries, 0 to 3989999
Data columns (total 37 columns):
idaviso                                int64
idpostulante                           object
GBA                                    bool
Full-time                              bool
senior                                 bool
junior                                 bool
otro                                   bool
Jefe                                   bool
chofer                                 bool
Ventas                                 bool
Comercial                              bool
Administración                         bool
Producción                             bool
Programación                           bool
Contabilidad                           bool
Tecnologia / Sistemas                  bool
Atención al Cliente                    bool
Mantenimiento                          bool
Recursos Humanos                       bool
postulacion                            bool
vis

In [99]:
test = pd.read_csv('data/test_final_100k.csv')

In [100]:
test = test.merge(usuarios, on='idpostulante', how = 'left')
test.head()

,id,idaviso,idpostulante,edad,sexo_masculino,titulo_universitario,titulo_secundario,titulo_superior,cantidad_postulaciones,postulaciones_Ventas,postulaciones_Comercial,postulaciones_Administración,postulaciones_Producción,postulaciones_Programación,postulaciones_Contabilidad,postulaciones_Tecnologia / Sistemas,postulaciones_Atención al Cliente,postulaciones_Mantenimiento,postulaciones_Recursos Humanos
0,0,739260,6M9ZQR,42.0,False,True,False,True,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,739260,6v1xdL,30.0,True,False,False,False,68.0,4.0,5.0,0.0,2.0,0.0,0.0,0.0,14.0,0.0,0.0
2,2,739260,ezRKm9,36.0,False,True,False,False,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,3,758580,1Q35ej,68.0,True,True,False,True,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,758580,EAN4J6,32.0,False,False,False,False,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [102]:
test = test.merge(avisos.drop(columns = 'nombre_area'),
                            on = 'idaviso',
                            how = 'left')
test.head()

,id,idaviso,idpostulante,edad,sexo_masculino,titulo_universitario,titulo_secundario,titulo_superior,cantidad_postulaciones,postulaciones_Ventas,...,Ventas,Comercial,Administración,Producción,Programación,Contabilidad,Tecnologia / Sistemas,Atención al Cliente,Mantenimiento,Recursos Humanos
0,0,739260,6M9ZQR,42.0,False,True,False,True,2.0,0.0,...,False,True,False,False,False,False,False,False,False,False
1,1,739260,6v1xdL,30.0,True,False,False,False,68.0,4.0,...,False,True,False,False,False,False,False,False,False,False
2,2,739260,ezRKm9,36.0,False,True,False,False,1.0,0.0,...,False,True,False,False,False,False,False,False,False,False
3,3,758580,1Q35ej,68.0,True,True,False,True,0.0,0.0,...,False,False,False,False,False,False,True,False,False,False
4,4,758580,EAN4J6,32.0,False,False,False,False,1.0,0.0,...,False,False,False,False,False,False,True,False,False,False


In [103]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 36 columns):
id                                     100000 non-null int64
idaviso                                100000 non-null int64
idpostulante                           100000 non-null object
edad                                   100000 non-null float64
sexo_masculino                         100000 non-null bool
titulo_universitario                   100000 non-null bool
titulo_secundario                      100000 non-null bool
titulo_superior                        100000 non-null bool
cantidad_postulaciones                 100000 non-null float64
postulaciones_Ventas                   100000 non-null float64
postulaciones_Comercial                100000 non-null float64
postulaciones_Administración           100000 non-null float64
postulaciones_Producción               100000 non-null float64
postulaciones_Programación             100000 non-null float64
postulaciones_Contabilida

In [104]:
test = test.merge(concatVistas, on = ['idaviso','idpostulante'], how = 'left')
test.head()

,id,idaviso,idpostulante,edad,sexo_masculino,titulo_universitario,titulo_secundario,titulo_superior,cantidad_postulaciones,postulaciones_Ventas,...,Comercial,Administración,Producción,Programación,Contabilidad,Tecnologia / Sistemas,Atención al Cliente,Mantenimiento,Recursos Humanos,visita
0,0,739260,6M9ZQR,42.0,False,True,False,True,2.0,0.0,...,True,False,False,False,False,False,False,False,False,NaN
1,1,739260,6v1xdL,30.0,True,False,False,False,68.0,4.0,...,True,False,False,False,False,False,False,False,False,NaN
2,2,739260,ezRKm9,36.0,False,True,False,False,1.0,0.0,...,True,False,False,False,False,False,False,False,False,NaN
3,3,758580,1Q35ej,68.0,True,True,False,True,0.0,0.0,...,False,False,False,False,False,True,False,False,False,NaN
4,4,758580,EAN4J6,32.0,False,False,False,False,1.0,0.0,...,False,False,False,False,False,True,False,False,False,NaN


In [105]:
test['visita'].fillna(False, inplace = True)
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 37 columns):
id                                     100000 non-null int64
idaviso                                100000 non-null int64
idpostulante                           100000 non-null object
edad                                   100000 non-null float64
sexo_masculino                         100000 non-null bool
titulo_universitario                   100000 non-null bool
titulo_secundario                      100000 non-null bool
titulo_superior                        100000 non-null bool
cantidad_postulaciones                 100000 non-null float64
postulaciones_Ventas                   100000 non-null float64
postulaciones_Comercial                100000 non-null float64
postulaciones_Administración           100000 non-null float64
postulaciones_Producción               100000 non-null float64
postulaciones_Programación             100000 non-null float64
postulaciones_Contabilida

# Empieza perceptron


In [116]:
datos = postulaciones.drop(columns = ['idpostulante','idaviso','postulacion'])
categorias = postulaciones['postulacion']

In [137]:
datos.head()

,GBA,Full-time,senior,junior,otro,Jefe,chofer,Ventas,Comercial,Administración,...,postulaciones_Ventas,postulaciones_Comercial,postulaciones_Administración,postulaciones_Producción,postulaciones_Programación,postulaciones_Contabilidad,postulaciones_Tecnologia / Sistemas,postulaciones_Atención al Cliente,postulaciones_Mantenimiento,postulaciones_Recursos Humanos
0,True,False,False,False,True,False,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,True,False,False,False,False,False,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,True,True,False,False,False,False,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,True,True,False,False,False,False,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,True,True,False,False,False,False,False,False,False,False,...,0.0,0.0,4.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0


In [193]:
test_size = 0.4
random_state = 0

In [164]:
datos_train, datos_test, cat_train, cat_test = train_test_split(datos,
                                                               categorias,
                                                               test_size = test_size,
                                                               random_state = random_state)

In [180]:
sc = StandardScaler()
sc.fit(datos_train)

datos_train_std = sc.transform(datos_train)
datos_test_std = sc.transform(datos_test)

In [181]:
#data_train = datos_train_std[:,0:5]
#data_test = datos_test_std[:,0:5]
n_iter=40
eta0 = 0.1

In [ ]:
#ppn1 features del 0 al 5

ppn1 = Perceptron(n_iter = n_iter,
                eta0 = eta0,
                random_state = random_state)

ppn1.fit(data_train,cat_train)



In [ ]:
cat_pred = ppn1.predict(data_test)

print('accuracy: {0:.2f}%'.format(accuracy_score(cat_test,cat_pred)*100))

In [182]:
#ppn2 con todos los features
ppn2 = Perceptron(n_iter = n_iter,
                eta0 = eta0,
                random_state = random_state)

ppn2.fit(datos_train_std,cat_train)



/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


Perceptron(alpha=0.0001, class_weight=None, eta0=0.1, fit_intercept=True,
      max_iter=None, n_iter=40, n_jobs=1, penalty=None, random_state=0,
      shuffle=True, tol=None, verbose=0, warm_start=False)

In [183]:
cat_pred = ppn2.predict(datos_test_std)

print('accuracy: {0:.2f}%'.format(accuracy_score(cat_test,cat_pred)*100))

accuracy: 73.80%


In [124]:
final_test_data = test[datos.columns]

In [125]:
final_test_data.head()

,GBA,Full-time,senior,junior,otro,Jefe,chofer,Ventas,Comercial,Administración,...,postulaciones_Ventas,postulaciones_Comercial,postulaciones_Administración,postulaciones_Producción,postulaciones_Programación,postulaciones_Contabilidad,postulaciones_Tecnologia / Sistemas,postulaciones_Atención al Cliente,postulaciones_Mantenimiento,postulaciones_Recursos Humanos
0,False,True,False,False,False,True,False,False,True,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,False,True,False,False,False,True,False,False,True,False,...,4.0,5.0,0.0,2.0,0.0,0.0,0.0,14.0,0.0,0.0
2,False,True,False,False,False,True,False,False,True,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,False,True,False,False,True,False,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,False,True,False,False,True,False,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [126]:
final_test_data = sc.transform(final_test_data)

In [184]:
prediccion = ppn2.predict(final_test_data)
prediccion = prediccion.astype(int)

In [185]:
test['sepostulo'] = prediccion
entregable = test[['id','sepostulo']]

In [186]:
entregable.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 2 columns):
id           100000 non-null int64
sepostulo    100000 non-null int64
dtypes: int64(2)
memory usage: 2.3 MB


In [187]:
entregable.index = entregable['id']
entregable.drop(columns = 'id', inplace = True)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [188]:
entregable.to_csv('submits/submit_7.csv')

In [191]:
entregable2 = pd.read_csv('submits/submit_6.csv')

In [192]:
print('accuracy: {0:.2f}%'.format(accuracy_score(entregable['sepostulo'],entregable2['sepostulo'])*100))

accuracy: 100.00%
